In [2]:
# Custom extra width cells
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%matplotlib inline
import requests,pandas as pd
 

In [4]:
# Get the key
with open(f'census_key.txt') as key:
    api_key=key.read().strip()

api_key = ''
year='2018'
dsource='pep'
dname='components'
cols='GEONAME,NATURALINC,DOMESTICMIG,INTERNATIONALMIG'
state='42'
county='017,029,045,091,101'

In [9]:
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

data_url = f'{base_url}?get={cols}&for=county:{county}&in=state:{state}&key={api_key}'
data_url = f'{base_url}?get={cols}&for=zipcode:*&in=state:{state}&key={api_key}'

response=requests.get(data_url)
print(response.text)

error: unknown/unsupported geography heirarchy


In [10]:
data = response.json()

df=pd.DataFrame(data[1:], columns=data[0]).\
    rename(columns={"NATURALINC": "Natural Increase", "DOMESTICMIG": "Net Domestic Mig",
                    "INTERNATIONALMIG":"Net Foreign Mig"})
df['fips']=df.state+df.county
df.set_index('fips', inplace=True)
df.drop(columns=['state','county'], inplace=True)
df = df.astype(dtype={'Natural Increase':'int64','Net Domestic Mig':'int64','Net Foreign Mig':'int64'})
df

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
year='2016'
dsource='zbp'
cols='ESTAB,EMP,PAYQTR1,PAYANN'
state='10'
 
base_url = f'https://api.census.gov/data/{year}/{dsource}'
 
data_url = f'{base_url}?get={cols}&for=zipcode:*&ST={state}&key={api_key}'
response=requests.get(data_url)
# print(response.text)

In [12]:
PLACEdata=response.json()
zbp_data=pd.DataFrame(data[1:], columns=data[0])
zbp_data.drop(columns=['ST'],inplace=True)
for field in cols.split(','):
    zbp_data=zbp_data.astype(dtype={field:'int64'})
zbp_data.head()

NameError: name 'data' is not defined

In [11]:
# https://atcoordinates.info/2019/09/24/examples-of-using-the-census-bureaus-api-with-python/
# https://towardsdatascience.com/getting-census-data-in-5-easy-steps-a08eeb63995d
# https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf
# https://api.census.gov/data/2018/acs/acs5/variables.html

year='2018'
dsource='acs' # American Cummunity Survey
dname='acs5' # 55 year survey

# B19001_001 -> HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)
cols='B19001_001E'


# https://api.census.gov/data/2018/acs/acs5?key=&get=B19001_001E,NAME,COUNTY,GEO_ID&for=zip%20code%20tabulation%20area:*%20state:*
geography_zip = 'zip%20code%20tabulation%20area:*'



In [12]:
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
# url_zip = "https://api.census.gov/data/2017/acs/acs5?get=NAME,group(B19013)&for={geography}"

    
# &for=zip%20code%20tabulation%20area:*"
data_url = f'{base_url}?key={api_key}&get={cols}&for={geography_zip}'

print(data_url)
response_by_zip = requests.get(data_url)
print(response_by_zip.text)


https://api.census.gov/data/2018/acs/acs5?key=&get=B19001_001E&for=zip%20code%20tabulation%20area:*
[["B19001_001E","zip code tabulation area"],
["3811","43964"],
["19911","28216"],
["26695","28277"],
["9176","28278"],
["13207","28303"],
["23124","28314"],
["217","28330"],
["524","28347"],
["1873","28351"],
["1065","28357"],
["1063","28369"],
["2386","28371"],
["590","28392"],
["2956","28428"],
["2083","28433"],
["945","28435"],
["989","28450"],
["8631","28461"],
["5394","28462"],
["57","28537"],
["2085","28547"],
["50","28552"],
["6897","28557"],
["11575","28560"],
["16195","28562"],
["8717","28570"],
["6345","28574"],
["242","28579"],
["1059","28373"],
["5117","28364"],
["2239","28366"],
["4214","28377"],
["2932","28383"],
["1868","28386"],
["2853","28398"],
["3946","28425"],
["2885","28429"],
["1145","28464"],
["2040","28073"],
["1252","28098"],
["1173","28119"],
["9491","28144"],
["10733","28150"],
["3768","28168"],
["17254","28173"],
["19859","28205"],
["14018","28214"],
["4840","

In [68]:
repsonse_data = response_by_zip

# Read the data to json
data = repsonse_data.json()

# Create the df
df_inc = pd.DataFrame(data[1:], columns=data[0]).\
    rename(columns={"B19001_001E": "Avg. Houshold inc. (infl)", 
                    "zip code tabulation area": "zipcode_ta"})

# Convert to integer so we can match it with the place/state file indexes
df_inc['zipcode_ta'] = df_inc['zipcode_ta'].astype(int)
df_inc.set_index('zipcode_ta', inplace=True)
df_inc.dropna(inplace=True)


# Sort before merging
# df_inc.sort_index(inplace=True)
# df_geoids.sort_index(inplace=True)

# Ouput the results
display(df_inc.head())

,Avg. Houshold inc. (infl)
zipcode_ta,
43964,3811
28216,19911
28277,26695
28278,9176
28303,13207


### Adding coordinates to each zipcode

In [61]:
def get_nan_col_stats(df_missing):
    # Get the missing values per each column 
    missing_series = df_missing.isna().sum()

    # Concatunate each column's type so we have a better idea for imputation
    missing_types_df = pd.concat([missing_series.rename('Missing'), df_missing.dtypes.rename('Dtype')], axis=1)
    missing_types_df.sort_values(by=['Missing'], ascending=False, inplace=True)
    display(missing_types_df)

In [74]:
# Data dir path from root of project
data_dir = "../data"

# http://download.geonames.org/export/zip/
dtypes_dict = {
    0: str, # country code  
    1: str,  # postal code 
    2: str,  # place name 
    
    3: str,  # admin name1  order subdivision (state) varchar(100)
    4: str, # admin code  subdivision (state) varchar(20)
    5: str, # admin name2  subdivision (county/province) varchar(100)
    6: str, # admin code  (county/province) varchar(20)
    7: str, # admin name3 (community) varchar(100)
    8: str, # admin code3  (community) varchar(20)
    
    9: float, # latitude
    10: float, # longitude
    11: float, # accuracy of lat/lng
}

us_zip_df = pd.read_csv(f"{data_dir}/US_zip.txt", sep="\t", 
            usecols=(0, 1, 2, 3, 4, 5, 6,  9, 10, 11), index_col=0,
            names=('country_code', 'postal_code', 'place_name', 
                   'state', 'state_code', 'county_sub', 'county_code',
                   'latitude', 'longitude', 'acc_lat_long'), dtype=dtypes_dict)

us_zip_df.reset_index(inplace=True)


# We only want to keep the coordinates and their accuracy
us_zip_df = us_zip_df[["postal_code", "latitude", "longitude", "acc_lat_long"]]
us_zip_df['zipcode_ta'] = us_zip_df['postal_code'].astype(int)
us_zip_df.dropna(inplace=True)

# # Set index to zipcode for merging
us_zip_df.set_index('zipcode_ta', inplace=True)


# Validate the output
display(us_zip_df.head())

get_nan_col_stats(us_zip_df)





,postal_code,latitude,longitude,acc_lat_long
zipcode_ta,,,,
99553,99553,54.1430,-165.7854,1.0
99571,99571,55.1858,-162.7211,1.0
99583,99583,54.8542,-163.4113,1.0
99612,99612,55.0628,-162.3056,1.0
99661,99661,55.3192,-160.4914,1.0


,Missing,Dtype
postal_code,0,object
latitude,0,float64
longitude,0,float64
acc_lat_long,0,float64


In [76]:
get_nan_col_stats(df_inc)

display(us_zip_df.head())
display(df_inc.head())

us_zip_df.index = us_zip_df.index.astype(int)
df_inc.index = df_inc.index.astype(int)


print(us_zip_df.index)
print(df_inc.index)

# Add coordinates to the zipcodes by merging
df_final = df_inc.merge(us_zip_df, left_index=True, right_index=True, how='left')
df_final.dropna(inplace=True)

# Remove any non matching column
# df_final = df_final.dropna()


# # df_inc
# df_final = df_inc.join(us_zip_df)

# df = pd.concat([df1,df2,df3])


# df1.add(df2, fill_value=0)

# Ouput results
print(len(df_inc.index))
print(len(us_zip_df.index))
print(len(df_final.index))
get_nan_col_stats(df_final)






,Missing,Dtype
Avg. Houshold inc. (infl),0,object


,postal_code,latitude,longitude,acc_lat_long
zipcode_ta,,,,
99553,99553,54.1430,-165.7854,1.0
99571,99571,55.1858,-162.7211,1.0
99583,99583,54.8542,-163.4113,1.0
99612,99612,55.0628,-162.3056,1.0
99661,99661,55.3192,-160.4914,1.0


,Avg. Houshold inc. (infl)
zipcode_ta,
43964,3811
28216,19911
28277,26695
28278,9176
28303,13207


Int64Index([99553, 99571, 99583, 99612, 99661, 99546, 99547, 99591, 99638,
            99660,
            ...
            96507, 96508, 96509, 96510, 96512, 96513, 96556, 96558, 96860,
            96863],
           dtype='int64', name='zipcode_ta', length=40938)
Int64Index([43964, 28216, 28277, 28278, 28303, 28314, 28330, 28347, 28351,
            28357,
            ...
            98134, 98174, 98222, 98233, 98243, 98279, 98280, 98311, 98326,
            98332],
           dtype='int64', name='zipcode_ta', length=33085)
33085
40938
32939


,Missing,Dtype
Avg. Houshold inc. (infl),0,object
postal_code,0,object
latitude,0,float64
longitude,0,float64
acc_lat_long,0,float64


In [30]:
# All relationdship files: https://www2.census.gov/geo/pdfs/maps-data/data/rel/
# https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_place_rel_10.txt

# https://opendata.stackexchange.com/a/13731
# Read in the ZCTA5 to PLACE & STATE file
zcta_place_relations_df = pd.read_csv("zcta_place_rel_10.txt")

# Set to index so we can match it
zcta_place_relations_df.set_index('ZCTA5', inplace=True)

# Only keep the columns we want to merge
zcta_place_best = zcta_place_relations_df[["STATE", "PLACE"]]

print(f"Place list amount -> {len(zcta_place_best.index)}")
display(zcta_place_best.head())


Place list amount -> 50404


,STATE,PLACE
ZCTA5,,
601,72,358
602,72,616
602,72,47873
603,72,745
603,72,9689


In [31]:
# Test a match 28277
zcta_place_best.loc[zcta_place_best.index == 28277]

# Make sure indexes are of the same type
display(zcta_place_best.index)
display(df_inc.index)


Int64Index([  601,   602,   602,   603,   603,   603,   606,   610,   610,
              610,
            ...
            99921, 99921, 99921, 99922, 99923, 99925, 99926, 99927, 99927,
            99929],
           dtype='int64', name='ZCTA5', length=50404)

Int64Index([43964, 28216, 28277, 28278, 28303, 28314, 28330, 28347, 28351,
            28357,
            ...
            98134, 98174, 98222, 98233, 98243, 98279, 98280, 98311, 98326,
            98332],
           dtype='int64', name='zipcode_ta', length=33120)

In [32]:
# Add state and county
df_inc2 = pd.merge(df_inc, zcta_place_best, left_index=True, right_index=True)
display(df_inc2)




# # 7200358
# print(f"Final list amount -> {len(df_final.index)}")
# print(zcta_place_best.loc[zcta_place_best['STATE'] == 72])
# print(df_final.loc[df_final['STATE'] == 72])

# # ZCTA5,STATE,PLACE,CLASSFP,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,ZAREA,ZAREALAND,PLPOP,PLHU,PLAREA,PLAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,PLPOPPCT,PLHUPCT,PLAREAPCT,PLAREALANDPCT


,Avg. Houshold inc. (infl),NAME,gid,STATE,PLACE
601,5517,ZCTA5 00601,8600000US00601,72,358
602,12738,ZCTA5 00602,8600000US00602,72,616
602,12738,ZCTA5 00602,8600000US00602,72,47873
603,19233,ZCTA5 00603,8600000US00603,72,745
603,19233,ZCTA5 00603,8600000US00603,72,9689
...,...,...,...,...,...
99925,325,ZCTA5 99925,8600000US99925,2,40400
99926,480,ZCTA5 99926,8600000US99926,2,48870
99927,29,ZCTA5 99927,8600000US99927,2,61190
99927,29,ZCTA5 99927,8600000US99927,2,63870


In [139]:
# All relationdship files: https://www2.census.gov/geo/pdfs/maps-data/data/rel/
# https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_place_rel_10.txt

# https://opendata.stackexchange.com/a/13731
# Read in the ZCTA5 to PLACE & STATE file
zcta_place_relations_df = pd.read_csv("zcta_place_rel_10.txt")
zcta_place_relations_df.set_index('ZCTA5', inplace=True)
# display(zcta_place_relations_df)

# Read in the ZCTA5 to County & STATE file
zcta_county_relations_df = pd.read_csv("zcta_county_rel_10.txt")
zcta_county_relations_df.set_index('ZCTA5', inplace=True)
# display(zcta_county_relations_df)

# Get the subsets we're interested in
zcta_place_best = zcta_place_relations_df[["STATE", "PLACE"]]
zcta_county_best = zcta_county_relations_df[["COUNTY"]]
display(zcta_county_relations_df.head())
display(zcta_place_best.head())

print(f"Place list amount -> {len(zcta_place_best.index)}")
print(f"County list amount -> {len(zcta_county_best.index)}")


# # Add zip and place id columns
df_final = pd.merge(zcta_place_best, zcta_county_best, left_index=True, right_index=True)
columns_ordered = ['PLACE', 'COUNTY', 'STATE']
df_final = df_final.reindex(columns=columns_ordered)

display(zcta_place_best)
display(df_final)

# df_final.isna().sum()

# 7200358
print(f"Final list amount -> {len(df_final.index)}")
# print(zcta_place_best.loc[zcta_place_best['STATE'] == 72])
# print(df_final.loc[df_final['STATE'] == 72])

# # ZCTA5,STATE,PLACE,CLASSFP,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,ZAREA,ZAREALAND,PLPOP,PLHU,PLAREA,PLAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,PLPOPPCT,PLHUPCT,PLAREAPCT,PLAREALANDPCT



,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,ZAREA,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
ZCTA5,,,,,,,,,,,,,,,,,,,,,
601,72,1,72001,18465,7695,165132671,164333375,18570,7744,167459085,...,173777444,172725651,99.43,99.37,98.61,98.60,94.77,94.71,95.03,95.14
601,72,141,72141,105,49,2326414,2326414,18570,7744,167459085,...,298027589,294039825,0.57,0.63,1.39,1.40,0.32,0.35,0.78,0.79
602,72,3,72003,41520,18073,83734431,79288158,41520,18073,83734431,...,117948080,79904246,100.00,100.00,100.00,100.00,98.95,98.99,70.99,99.23
603,72,5,72005,54689,25653,82063867,81880442,54689,25653,82063867,...,195741178,94608641,100.00,100.00,100.00,100.00,89.73,90.23,41.92,86.55
606,72,93,72093,6276,2740,94864349,94851862,6615,2877,109592548,...,94864349,94851862,94.88,95.24,86.56,86.56,100.00,100.00,100.00,100.00


,STATE,PLACE
ZCTA5,,
601,72,358
602,72,616
602,72,47873
603,72,745
603,72,9689


Place list amount -> 50404
County list amount -> 44410


,STATE,PLACE
ZCTA5,,
601,72,358
602,72,616
602,72,47873
603,72,745
603,72,9689
...,...,...
99925,2,40400
99926,2,48870
99927,2,61190


,PLACE,COUNTY,STATE
ZCTA5,,,
601,358,1,72
601,358,141,72
602,616,3,72
602,47873,3,72
603,745,5,72
...,...,...,...
99925,40400,198,2
99926,48870,198,2
99927,61190,198,2


Final list amount -> 69642


In [109]:

# display(df_inc.describe())

display(df_geoids.head())
display(df_geoids.describe())


# df_inc['State'] = df_inc['gid'].map(df_geoids['State'])
# display(df_inc.head())

# display(df_inc.describe())



# Add state and county
# df_final = pd.merge(df_inc, df_geoids, left_index=True, right_index=True)
# display(df_inc.join(df_geoids))



,Place_state,State_id,Place_id,Place,State
gid,,,,,
1600000US2967718,"Sibley village, Missouri",29,67718,Sibley village,Missouri
1600000US2839640,"Laurel city, Mississippi",28,39640,Laurel city,Mississippi
1600000US2840280,"Leland city, Mississippi",28,40280,Leland city,Mississippi
1600000US2815700,"Corinth city, Mississippi",28,15700,Corinth city,Mississippi
1600000US2971440,"Sullivan city, Missouri",29,71440,Sullivan city,Missouri


,Place_state,State_id,Place_id,Place,State
count,29573,29573,29573,29573,29573
unique,29573,52,19508,24507,54
top,"Town of Pines town, Indiana",42,00100,Franklin city,Pennsylvania
freq,1,1761,14,16,1761


In [102]:
# https://atcoordinates.info/2019/09/24/examples-of-using-the-census-bureaus-api-with-python/
# https://towardsdatascience.com/getting-census-data-in-5-easy-steps-a08eeb63995d
# https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf
# https://api.census.gov/data/2018/acs/acs5/variables.html

year='2018'
dsource='acs' # American Cummunity Survey
dname='acs5' # 5 year survey

# B19001_001 -> HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)
cols='NAME,GEO_ID'
geography_state = 'place:*'
# https://api.census.gov/data/2018/acs/acs5?key=&get=B19001_001E,NAME,COUNTY,GEO_ID&for=zip%20code%20tabulation%20area:*%20state:*


base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
# url_zip = "https://api.census.gov/data/2017/acs/acs5?get=NAME,group(B19013)&for={geography}"

    
# &for=zip%20code%20tabulation%20area:*"
data_url = f'{base_url}?key={api_key}&get={cols}&for={geography_state}'

print(data_url)
response_geos = requests.get(data_url)
print(response_geos.text)


https://api.census.gov/data/2018/acs/acs5?key=&get=NAME,GEO_ID&for=place:*
[["NAME","GEO_ID","state","place"],
["Sibley village, Missouri","1600000US2967718","29","67718"],
["Laurel city, Mississippi","1600000US2839640","28","39640"],
["Leland city, Mississippi","1600000US2840280","28","40280"],
["Corinth city, Mississippi","1600000US2815700","28","15700"],
["Sullivan city, Missouri","1600000US2971440","29","71440"],
["Lewistown town, Missouri","1600000US2941852","29","41852"],
["Sugar Creek city, Missouri","1600000US2971368","29","71368"],
["La Grange city, Missouri","1600000US2939764","29","39764"],
["Livonia village, Missouri","1600000US2943454","29","43454"],
["Hunnewell city, Missouri","1600000US2933760","29","33760"],
["Center city, Missouri","1600000US2912592","29","12592"],
["Puxico city, Missouri","1600000US2960284","29","60284"],
["Chamois city, Missouri","1600000US2913060","29","13060"],
["Linn city, Missouri","1600000US2943238","29","43238"],
["Doniphan city, Missouri","160

In [104]:
# Read the data to json
data = response_geos.json()

# Create the df
df_geoids = pd.DataFrame(data[1:], columns=data[0]).\
    rename(columns={"NAME": "Place_state", "state": "State_id",
                    "place": "Place_id", "GEO_ID":"gid"})


# https://stackoverflow.com/a/61313413/8970591
df_geoids[['Place', 'State']] = df_geoids['Place_state'].str.split(',', n=1, expand=True)
# df_geoids.drop(columns=['County_state', 'State_id', 'County_id'], inplace=True)
df_geoids.set_index('gid', inplace=True)

# Ouput the results
display(df_geoids.head())

,Place_state,State_id,Place_id,Place,State
gid,,,,,
1600000US2967718,"Sibley village, Missouri",29,67718,Sibley village,Missouri
1600000US2839640,"Laurel city, Mississippi",28,39640,Laurel city,Mississippi
1600000US2840280,"Leland city, Mississippi",28,40280,Leland city,Mississippi
1600000US2815700,"Corinth city, Mississippi",28,15700,Corinth city,Mississippi
1600000US2971440,"Sullivan city, Missouri",29,71440,Sullivan city,Missouri


In [80]:
df_geoids.loc[df_geoids.index == '0500000US43964']





,County,State
gid,,


### Matching GEO_ID to state and county names
We downloaded the full geoid US.txt file [geonames.org](http://download.geonames.org/export/dump/)

In [25]:
import csv


# Right mapping from: https://stackoverflow.com/questions/34498417/import-text-file-from-geonames-using-pandas-python
dtypes_dict = {
    0: int, # geonameid
    1: str,  # name
    2: str,  # asciiname
    3: str,  # alternatenames
    4: float, # latitude
    5: float, # longitude
    6: str, # feature class
    7: str, # feature code
    8: str, # country code
    9: str, # cc2
    10: str, # admin1 code
    11: str, # admin2 code
    12: str, # admin3 code
    13: str, # admin4 code
    14: int, # population
    15: float, # elevation
    16: int, # dem (digital elevation model)
    17: str, # timezone
    18: str # modification date yyyy-MM-dd
}

df_geoids = pd.read_csv("US.txt", sep="\t", 
            usecols=(0, 1, 2, 3, 10, 11), index_col=0,
            names=('gid', 'name', 'asciiname', 'alternatenames',
                   'fipscode', 'county'), dtype=dtypes_dict)
df_geoids.head()




/usr/local/anaconda3/envs/cs109a/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,name,asciiname,alternatenames,fipscode,county
gid,,,,,
2130833,McArthur Reef,McArthur Reef,NaN,AK,016
3577483,The Narrows,The Narrows,The Narrows,00,NaN
3831601,Nantucket Shoals,Nantucket Shoals,NaN,MA,NaN
3831610,Little Stellwagen Basin,Little Stellwagen Basin,NaN,00,NaN
3831661,Cashes Ledge,Cashes Ledge,"Cash Ledge,Cashe Ledge",NaN,NaN


In [29]:
df_geoids.loc[df_geoids.index == 4835000]


,name,asciiname,alternatenames,fipscode,county
gid,,,,,
4835000,Glasgo Post Office,Glasgo Post Office,NaN,CT,011


In [24]:
zip_data_json = response_by_zip.json()

# cols='GEONAME,NATURALINC,DOMESTICMIG,INTERNATIONALMIG'

new_colnames = {"B19001_001E": "Avg. Houshold Income (inf adj)", "zip code tabulation area":"Zip code",
                "PLACE": "Place", "REGION":"Region", "STNAME": "State"}

df = pd.DataFrame(zip_data_json[1:], columns=zip_data_json[0]).rename(columns=new_colnames)

df.set_index("Zip code", inplace=True)
# df.drop(columns=['state','county'], inplace=True)
# df = df.astype(dtype={'Natural Increase':'int64','Net Domestic Mig':'int64','Net Foreign Mig':'int64'})
df

,Avg. Houshold Income (inf adj),NAME,Place,Region,STATE
Zip code,,,,,
43964,3811,ZCTA5 43964,None,None,None
28216,19911,ZCTA5 28216,None,None,None
28277,26695,ZCTA5 28277,None,None,None
28278,9176,ZCTA5 28278,None,None,None
28303,13207,ZCTA5 28303,None,None,None
...,...,...,...,...,...
98279,340,ZCTA5 98279,None,None,None
98280,238,ZCTA5 98280,None,None,None
98311,10146,ZCTA5 98311,None,None,None
